In [19]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers

In [20]:
training_dir = '/Users/kgraham/data/dog-cat-images/small_train'
test_dir = '/Users/kgraham/data/dog-cat-images/small_test'


## Loading the images

In [21]:
img_width, img_height = 150, 150

# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        training_dir,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary')

test_generator = datagen.flow_from_directory(
        test_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

Found 4096 images belonging to 2 classes.
Found 512 images belonging to 2 classes.


# The Model

In [24]:
model = Sequential() # container for the layers

# layer 1: 32 3x3 filters, 2x2 max pooling layer
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# layer 2: identical to layer 1
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# layer 3: 64 3x3 filters
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) # flatten feature map to 1D
model.add(Dense(64)) # fully-connected layer of 64 nodes
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1)) # single node
model.add(Activation('sigmoid')) # gives us a probability

In [25]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop', 
              metrics=['accuracy'])

## Training the model

In [30]:
epochs = 30
training_samples = 4096/2
test_samples = 512

In [31]:
model.fit_generator(
        train_generator,
        samples_per_epoch=training_samples,
        nb_epoch=epochs,
        validation_data=test_generator,
        nb_val_samples=test_samples)

Epoch 1/30
2048/2048 [==============================] - 67s - loss: 0.6439 - acc: 0.6318 - val_loss: 0.6505 - val_acc: 0.6172